In [1]:
import pandas as pd
import duckdb

176. Вторая по величине зарплата  
Напишите решение для поиска второй по величине уникальной заработной платы в Employeeтаблице. Если второй по величине заработной платы нет, верните значение  null (return None in Pandas).

In [2]:
data = [[1, 100], [2, 200], [3, 300]]
employee = pd.DataFrame(data, columns=['id', 'salary']).astype({'id':'int64', 'salary':'int64'})
employee.head()

,id,salary
0,1,100
1,2,200
2,3,300


In [17]:
data = [[1, 0], [2, 0], [3, 1]]
employee = pd.DataFrame(data, columns=['id', 'salary']).astype({'id':'int64', 'salary':'int64'})
employee.head()

,id,salary
0,1,0
1,2,0
2,3,1


In [ ]:
# мое решение
employee = employee.drop_duplicates(subset=['salary']) 
employee['rank'] = employee['salary'].rank(method='dense', ascending=False).astype('int64')
data_1 = [1,2]
ranke_1 = pd.DataFrame(data_1, columns=['ranke_1']).astype({'ranke_1':'int64'})
union_table = ranke_1.merge(employee, left_on='ranke_1', right_on='rank', how='left')
union_table[union_table['ranke_1'] == 2][['salary']].rename(columns={'salary': 'SecondHighestSalary'})

In [ ]:
# решение GPT
salaries = employee['salary'].drop_duplicates().sort_values(ascending=False)
result = (
    salaries.iloc[1:2]
    .to_frame(name='SecondHighestSalary')
)
if result.empty:
    result = pd.DataFrame({'SecondHighestSalary': [None]})

,SecondHighestSalary
0,0


In [19]:
query = """
with t1 as (
    select 
        salary,
        dense_rank() over(order by salary desc) as rank
    from employee
    )
select
    case when count(*) < 1 then null
         else max(salary) end as SecondHighestSalary
from t1
where rank > 1
"""
result = duckdb.query(query).to_df()
result

,SecondHighestSalary
0,0


177. N-я самая высокая зарплата  
Напишите решение для поиска самой высокой уникальной заработной платы в таблице. Если уникальных зарплат  меньше, чем указано , верните значение .nthEmployeennull

In [8]:
N = 2
col_name = f'getNthHighestSalary({N})'
salaries = employee['salary'].drop_duplicates().sort_values(ascending=False)
if N > 0 and len(salaries) >= N:
    result = salaries.iloc[N-1:N].to_frame(name=col_name)
else:
    result = pd.DataFrame({col_name: [None]})
result

,getNthHighestSalary(2)
1,200


In [ ]:
N = 2
query = """
with t1 as (
    select 
        e.salary,
        dense_rank() over(order by e.salary desc) as rank
    from employee e
    )
select
    case when count(*) < 1 or N <= 0 then null
        else max(t1.salary) end as SecondHighestSalary
from t1
where rank > N - 1
"""
result = duckdb.query(query).to_df()
result

178. Ранговые баллы  
Напишите решение для определения ранга оценок. Ранжирование должно рассчитываться в соответствии со следующими правилами:  
- Оценки следует располагать в порядке убывания.  
- Если между двумя результатами есть ничья, оба показателя должны иметь одинаковый рейтинг.  
- В случае ничьей следующий номер в рейтинге должен быть следующим целым числом. Другими словами, между рангами не должно быть разрывов.  
Возвращает таблицу результатов, отсортированную в scoreпорядке убывания.

In [4]:
data = [[1, 3.5], [2, 3.65], [3, 4.0], [4, 3.85], [5, 4.0], [6, 3.65]]
scores = pd.DataFrame(data, columns=['id', 'score']).astype({'id':'Int64', 'score':'Float64'})
scores

,id,score
0,1,3.5
1,2,3.65
2,3,4.0
3,4,3.85
4,5,4.0
5,6,3.65


In [7]:
query = """
select
    score,
    dense_rank() over (order by score desc) as rank
from 
scores
"""
result = duckdb.query(query).to_df()
result

,score,rank
0,4.00,1
1,4.00,1
2,3.85,2
3,3.65,3
4,3.65,3
5,3.50,4


In [11]:
scores['rank'] = scores['score'].rank(method='dense', ascending=False).astype('Int64')
scores.sort_values('rank')[['score', 'rank']]

,score,rank
2,4.0,1
4,4.0,1
3,3.85,2
1,3.65,3
5,3.65,3
0,3.5,4


180. Последовательные номера  
Найдите все числа, которые встречаются как минимум три раза подряд.

In [13]:
data = [[1, 1], [2, 1], [3, 1], [4, 2], [5, 1], [6, 2], [7, 2]]
logs = pd.DataFrame(data, columns=['id', 'num']).astype({'id':'Int64', 'num':'Int64'})
logs

,id,num
0,1,1
1,2,1
2,3,1
3,4,2
4,5,1
5,6,2
6,7,2


In [ ]:
# мое решение
numbers = logs['num']
if len(numbers) > 0:
    result = []
    a = numbers[0]
    b = 1
    for i in range(1, len(numbers)):
        c = numbers[i]
        if c == a:
            b = b + 1
        else:
            b = 1
        a = c
        print(b)
        if b >= 3:
            result.append(numbers[i])
            b = 1
else:
    result = []
result_df = pd.DataFrame(result, columns=['ConsecutiveNums'])
result_df[['ConsecutiveNums']].drop_duplicates()

In [ ]:
# улучшение моего решения через цикл
nums = logs['num'].tolist()
result = []
count = 1
for i in range(1, len(nums)):
    if nums[i] == nums[i-1]:
        count += 1
        if count == 3:
            result.append(nums[i])
    else:
        count = 1
pd.DataFrame({'ConsecutiveNums': result})

In [ ]:
# еще одно решение GPT, через pandas
result = logs.loc[
    (logs['num'] == logs['num'].shift(1)) &
    (logs['num'] == logs['num'].shift(-1)),
    'num'
].drop_duplicates()
pd.DataFrame({'ConsecutiveNums': result})

In [ ]:
query = """
with t1 as(
    select 
        num,
        lag(num, 1, 0) over (order by id) as prev_1,
        lag(num, 2, 0) over (order by id) as prev_2
    from logs)
select distinct num as ConsecutiveNums
from t1
where num = prev_1 and num = prev_2
"""
result = duckdb.query(query).to_df()
result

184. Напишите решение для поиска сотрудников с самой высокой заработной платой в каждом из отделов.

In [4]:
data = [[1, 'Joe', 70000, 1], [2, 'Jim', 90000, 1], [3, 'Henry', 80000, 2], [4, 'Sam', 60000, 2], [5, 'Max', 90000, 1]]
employee = pd.DataFrame(data, columns=['id', 'name', 'salary', 'departmentId']).astype({'id':'Int64', 'name':'object', 'salary':'Int64', 'departmentId':'Int64'})
data = [[1, 'IT'], [2, 'Sales']]
department = pd.DataFrame(data, columns=['id', 'name']).astype({'id':'Int64', 'name':'object'})

In [5]:
employee

,id,name,salary,departmentId
0,1,Joe,70000,1
1,2,Jim,90000,1
2,3,Henry,80000,2
3,4,Sam,60000,2
4,5,Max,90000,1


In [6]:
department

,id,name
0,1,IT
1,2,Sales


In [13]:
# мое решение 
employee['rank'] = employee.groupby('departmentId')['salary'].rank(ascending=False).astype('int64')
merged = employee.merge(department, how='left', left_on='departmentId', right_on='id')
merged[merged['rank'] == 1].rename(columns={
    'name_y': 'Department', 
    'name_x': 'Employee', 
    'salary': 'Salary'
    })[['Department', 'Employee', 'Salary']]

,Department,Employee,Salary
1,IT,Jim,90000
2,Sales,Henry,80000
4,IT,Max,90000


In [8]:
# решение с сайта
merged = employee.merge(department, left_on='departmentId', right_on='id', how='left')
highest_salary = merged.loc[merged.groupby('departmentId')['salary'].transform('max') == merged['salary']]
result = highest_salary[['name_x', 'salary', 'name_y']].rename(columns={ 
    'name_y': 'Department',  # department name column
    'name_x': 'Employee',    # employee name column
    'salary': 'Salary'       # salary column
})
result[['Department', 'Employee', 'Salary']]

,Department,Employee,Salary
1,IT,Jim,90000
2,Sales,Henry,80000
4,IT,Max,90000


In [10]:
query = """
with t1 as(
    select
        d.name as Department,
        e.name as Employee,
        e.salary as Salary,
        dense_rank() over(partition by departmentId order by salary desc) as salary_rank
    from employee e
    join department d
    on e.departmentId = d.id
)
select
    Department,
    Employee,
    Salary
from t1
where salary_rank = 1
"""
result = duckdb.query(query).to_df()
result

,Department,Employee,Salary
0,Sales,Henry,80000
1,IT,Jim,90000
2,IT,Max,90000


550. Напишите решение, которое выведет долю игроков, повторно вошедших в систему на следующий день после первого входа, округлив до двух знаков после запятой . Другими словами, вам нужно определить количество игроков, вошедших в систему на следующий день после первого входа, и разделить его на общее количество игроков.

In [2]:
data = [[1, 2, '2016-03-01', 5], [1, 2, '2016-03-02', 6], [2, 3, '2017-06-25', 1], [3, 1, '2016-03-02', 0], [3, 4, '2018-07-03', 5]]
activity = pd.DataFrame(data, columns=['player_id', 'device_id', 'event_date', 'games_played']).astype({'player_id':'Int64', 'device_id':'Int64', 'event_date':'datetime64[ns]', 'games_played':'Int64'})
activity.head()

,player_id,device_id,event_date,games_played
0,1,2,2016-03-01,5
1,1,2,2016-03-02,6
2,2,3,2017-06-25,1
3,3,1,2016-03-02,0
4,3,4,2018-07-03,5


In [ ]:
query = """
with t as(
    select
        player_id,
        min(event_date) as min_date
    from activity
    group by player_id)
select
    round(count(distinct t1.player_id) * 1.0 / (select count(distinct player_id) from activity),2) as fraction
from t t1 join activity t2
    on t1.min_date + 1 = t2.event_date and t1.player_id = t2.player_id
"""
result = duckdb.query(query).to_df()
result

In [41]:
activity['min_date'] = activity.groupby('player_id', as_index=False).event_date.transform(min)
pd.DataFrame([round(activity[activity['event_date'] == activity['min_date'] + pd.Timedelta(days=1)].player_id.nunique() / activity.player_id.nunique(),2)], columns=['fraction'])

C:\Users\user\AppData\Local\Temp\ipykernel_1916\3943309026.py:1: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  activity['min_date'] = activity.groupby('player_id', as_index=False).event_date.transform(min)


,fraction
0,0.33
